In [ ]:
import pandas as pd
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from itertools import combinations
from sklearn.cluster import KMeans
from collections import deque
import random

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

class DQNTSP(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQNTSP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

def train_rl_tsp(points, num_episodes=1000, gamma=0.99, lr=0.001):
    num_nodes = len(points)
    model = DQNTSP(num_nodes, num_nodes)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    memory = deque(maxlen=5000)
    batch_size = 64
    
    for episode in range(num_episodes):
        state = np.random.permutation(num_nodes)
        total_reward = 0
        
        for step in range(num_nodes - 1):
            current_node = state[step]
            next_node = state[step + 1]
            reward = -haversine(points[current_node][0], points[current_node][1], points[next_node][0], points[next_node][1])
            total_reward += reward
            memory.append((current_node, next_node, reward))
            
            if len(memory) > batch_size:
                minibatch = random.sample(memory, batch_size)
                for s, a, r in minibatch:
                    target = r + gamma * torch.max(model(torch.tensor([a], dtype=torch.float32)))
                    output = model(torch.tensor([s], dtype=torch.float32))[a]
                    loss = (target - output) ** 2
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
        
        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")
    
    return model

def tsp_optimized(model, points):
    num_nodes = len(points)
    state = np.random.permutation(num_nodes)
    optimized_route = [state[0]]
    
    for _ in range(num_nodes - 1):
        current_node = optimized_route[-1]
        action_values = model(torch.tensor([current_node], dtype=torch.float32)).detach().numpy()
        next_node = np.argmax(action_values)
        optimized_route.append(next_node)
    
    optimized_route.append(state[0])
    return optimized_route

def multi_slot_clustering(shipments, num_clusters=3):
    coords = np.array([[s['lat'], s['lon']] for s in shipments])
    kmeans = KMeans(n_clusters=min(num_clusters, len(shipments)), random_state=0).fit(coords)
    clusters = [[] for _ in range(kmeans.n_clusters)]
    
    for idx, label in enumerate(kmeans.labels_):
        clusters[label].append(shipments[idx])
    
    return clusters

# Load data (unchanged)
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
store_lat = store_df['Latitute']
store_lon = store_df['Longitude']
shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')

shipments = []
for _, row in shipments_df.iterrows():
    shipments.append({'id': row['Shipment ID'], 'lat': row['Latitude'], 'lon': row['Longitude']})

# Train RL model for TSP
points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in shipments]
tsp_model = train_rl_tsp(points)

# Cluster shipments into multi-slot groups
multi_slot_groups = multi_slot_clustering(shipments)

trips = []
for group in multi_slot_groups:
    points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in group]
    optimized_route = tsp_optimized(tsp_model, points)
    
    trips.append({'shipments': group, 'route': optimized_route})

print("Optimized Trips:")
for trip in trips:
    print(trip)


ModuleNotFoundError: No module named 'sklearn'